## Dataset preparation - Stage 2 - Using LMQG for dataset preparation (Cell phones and Accessories)

In [2]:
!pip install lmqg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.3 MB/s eta 0:00:00
  Created wheel for lmqg: filename=lmqg-0.1.1-py3-none-any.whl size=111599 sha256=752fe6ad9fd98209ae9811c35b454d15998d400ce122713c732a5edb0e208aad
  Stored in directory: /root/.cache/pip/wheels/a6/b1/8d/5ad3d68cf09f7d87bca431d42c429798096291744b0bd80f2d
  Created wheel for sudachidict_core: filename=SudachiDict_core-20230110-py3-none-any.whl size=71665396 sha256=721ef83ab79cae5f53f5691162ac18f1c4d077ce7ab55016ae4fb4c03be84efe
  Stored in directory: /root/.cache/pip/wheels/35/35/8a/5cd8203a86e68ccefc49d4a2975165bb9ee369d2693eb4049b
Successfully built lmqg sudachidict_core


In [3]:
import json
import pandas as pd
import os
import gzip
from pprint import pprint
from collections import Counter
from lmqg import TransformersQG

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
path_meta = "/kaggle/input/data-cell/Cell_Phones_and_Accessories_data-2.json" #add json file

with open(path_meta,'r') as file:
    data = json.load(file)

In [5]:
len(data) # task was divided into 5500, and 5000 samples with team mate.

5500

In [6]:
model1 = TransformersQG(model='lmqg/t5-base-squad-qg', model_ae='lmqg/t5-base-squad-ae')

In [7]:
#data to batches - Due to less computational resources we divied it in batches of 500 and then fed to model for prediction.
batch_1 = data[:500]
batch_2 = data[500:1000]
batch_3 = data[1000:1500]
batch_4 = data[1500:2000]
batch_5 = data[2000:2500]
batch_6 = data[2500:3000]
batch_7 = data[3000:3500]
batch_8 = data[3500:4000]
batch_9 = data[4000:4500]
batch_10 = data[4500:5000]
batch_11 = data[5000:5501]
data_1 = batch_11 #change batches here   
len(data_1)

500

In [8]:
test_data = []
for x in data_1:
    words = x['description'].split()
    ls = Counter([len(words[i]) == 1 for i in range(len(words) - 1)])
    if len(x['description'].split('.')) <= 30:
        if ls[1] < ls[0]:
            test_data.append(x)
        else:
            print(x,ls)
len(test_data)

499

In [9]:
# paragraph to generate pairs of question and answer
main_data = []
for d in test_data:
    dic_temp = {}
    dic_temp['id'] = d['id']
    dic_temp['title'] = d['title']
    dic_temp['context'] = d['description']
    context = d['description']
    question_answer = model1.generate_qa(list_context=context,num_questions=8,sentence_level=True)
    question_answer = list(dict.fromkeys(question_answer))
    qls = []
    for i in question_answer:
        loc = dic_temp['context'].find(i[1])
        if loc != -1:
            temp = {}
            ans = {}
            ans['answer_start'] = loc
            ans['text'] = i[1]
            temp['answer'] = ans
            temp['question'] = i[0]
            qls.append(temp)
    dic_temp['qas'] = qls
    main_data.append(dic_temp)


100%|██████████| 9/9 [00:00<00:00, 1954.07it/s]


In [10]:
main_data[-1] #for verification

{'id': 'B004IQM190',
 'title': 'Blue Vine Design Snap-On Cover Case for Metro PCS Samsung Messager 3 R570',
 'context': 'It is a Unknown product. This Metro PCS Samsung Messager R blue vine case cover provides excellent protection from dust scratches and unwanted blemishes. The Metro PCS Samsung Messager R blue vine case cover also allows for full functionality of your phone with openings for all buttons ports jacks and speakers. Provide your phone with excellent protection and give it a fashionable and attractive look at the same time with this Metro PCS Samsung Messager R blue vine case cover. Premium high quality snap on hard cover case protector. Designed to fit phone perfectly. Snaps right over your phone and gives it a new look while providing great protection. Openings for full phone functionality. Constructed for strong durability and scratch resistance. Categories of product are Cell Phones Accessories Cases Holsters Sleeves Basic Cases',
 'qas': [{'answer': {'answer_start': 8

In [11]:
len(main_data)

499

## Save batches

In [12]:
with open('Cell_Phones_and_Accessories_batch_11.json','w') as file:
    json.dump(main_data,file,indent=4)